In [1]:
import cv2
import os
import numpy as np
import yaml
from yaml.loader import SafeLoader

In [2]:
#load yaml
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader = SafeLoader)

labels = data_yaml['names']
print(labels)

['person', 'dog']


In [4]:
#load yolo
yolo = cv2.dnn.readNetFromONNX('./Model3/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [87]:
#laod the image
img = cv2.imread('./img_3.jpg')
image = img.copy()
row, col, d = img.shape

max_rc = max(row, col)
input_image = np.zeros((max_rc,max_rc,3),dtype = np.uint8)
input_image[0:row,0:col] = img

INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO), swapRB = True, crop=False)
yolo.setInput(blob)
preds = yolo.forward()


In [88]:
print(preds.shape)

(1, 25200, 7)


In [89]:
detections = preds[0]
boxes = []
confidences = []
classes = []

image_w, image_h = input_image.shape[:2]
x_factor = image_w / INPUT_WH_YOLO
y_factor = image_h / INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]
    if confidence > 0.4:
        class_score = row[:5].max()
        class_id = row[5:].argmax()
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            left = int((cx - 0.5 * w) * x_factor)
            top = int((cy - 0.5 * h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)
            box = np.array([left, top, width, height])
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

indices = cv2.dnn.NMSBoxes(np.array(boxes), np.array(confidences), 0.25, 0.45).flatten()
boxes_np = np.array(boxes)
confidence_np = np.array(confidences)


In [90]:
len(indices)

8

In [93]:
thickness = 2  # Example thickness value
for ind in indices:
    x, y, w, h = boxes[ind]
    bb_conf = confidences[ind]
    class_id = classes[ind]
    class_name = labels[class_id]
    text = f'{class_name}: {bb_conf * 100:.2f}%'
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), thickness)
    cv2.rectangle(image, (x, y-30), (x+w, y), (255, 255, 255), -1)
    cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)



In [94]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction', image)
cv2.waitKey(0)
cv2.destroyAllWindows()